In [184]:
# importing libraries
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [185]:
death_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/death.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/confirmed.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/recovered.csv')
provinces_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/provinces.csv')
case_per_provinces_df = pd.read_csv('https://raw.githubusercontent.com/OliverArthur/covid-19-rd-data/master/case-per-provinces.csv')

In [186]:
# clean data
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
provinces_df.columns = map(str.lower, provinces_df.columns)

In [187]:
confirmed_df.head()
cols = confirmed_df.keys()
confirmed = confirmed_df.loc[:, cols[3]:cols[-1]]
deaths = death_df.loc[:, cols[3]:cols[-1]]
recovered_list = recovered_df.loc[:, cols[3]:cols[-1]]

dates = confirmed.keys()
confirmed_total = []
deaths_total = []
recovered_total = []
for i in dates:
    confirmed_sum = confirmed[i].sum()
    deaths_sum = deaths[i].sum()
    recovered_sum = recovered_list[i].sum()
    confirmed_total.append(confirmed_sum)
    deaths_total.append(deaths_sum)
    recovered_total.append(recovered_sum)
    

In [188]:
display(HTML("<div style = 'background-color: #504e4e; padding: 30px '>" +
             "<span style='color: #fff; font-size:30px;'> Confirmed: "  + str(confirmed_sum) +"</span>" +
             "<span style='color: red; font-size:30px;margin-left:20px;'> Deaths: " + str(deaths_sum) + "</span>"+
             "<span style='color: lightgreen; font-size:30px; margin-left:20px;'> Recovered: " + str(recovered_sum) + "</span>"+
             "</div>")
       )



In [189]:
# number of provicens to enter
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

def show_latest_cases(n):
    n = int(n)
    return case_per_provinces_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [190]:
sorted_provinces_df = case_per_provinces_df.sort_values('confirmed', ascending= False)

KeyError: 'confirmed'

In [ ]:
# # plotting the 10 worst hit provinces
def bubble_chart(n):
    fig = px.scatter(sorted_provinces_df.head(n), x="provincia", y="confirmed", size="confirmed", color="pais",
               hover_name="provincia", size_max=60)
    fig.update_layout(
    title=str(n) +" Las provincias más afectados",
    xaxis_title="Provincias",
    yaxis_title="Casos confirmados",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

In [ ]:
def plot_cases_of_a_country(provincia):
    labels = ['confirmed', 'death']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_sum, deaths_sum]
    
    fig = go.Figure()
    
    for i, df in enumerate(df_list):
        if provincia == 'Distrito Nacional' or provincia == 'distrito nacional':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == provincia].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="Casos de COVID 19 en la provincia " + provincia,
        xaxis_title='Fecha',
        yaxis_title='No existen casos confirmado',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show()

In [ ]:
interact(plot_cases_of_a_country, provincia='Distrito Nacional')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

In [ ]:
px.bar(
    sorted_provinces_df.head(10),
    x = "provincia",
    y = "confirmed",
    title= "Top 10 provincias mas afectadas", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)


In [ ]:
px.bar(
    sorted_provinces_df.head(10),
    x = "provincia",
    y = "death",
    title= "Top 10 provincias con mas muertes", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [ ]:
px.bar(
    sorted_provinces_df.head(10),
    x = "provincia",
    y = "recovered",
    title= "Top 10 provincias con mas casos de recuperados", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [ ]:
RD_COORDINATES = (18.606258, -70.106722)
provices_map = folium.Map(location=RD_COORDINATES, tiles="cartodbpositron", zoom_start=6, max_zoom=12, min_zoom = 2)

provices_map